In [1]:
import sys
import neal
import greedy
import tabu
import time
import numpy as np

from pathlib import Path
current_path = Path().resolve()
sys.path.append(str(current_path / '../code/'))
from experiment import Experiment
from table import Table
from visualisation import Figure

# Load the data

In [2]:
data_folder = '../data/Quadratic Assignment Problem/'

file_names = ['had12.npz', 'had14.npz', 'had16.npz',
             'had18.npz', 'had20.npz', 'rou12.npz',
             'rou15.npz', 'rou20.npz', 'tai40a.npz',
             'tai40b.npz']

loaded_files = [np.load(data_folder + file) for file in file_names]

obj_qubos = [i['cost_function_qubo'] for i in loaded_files]
obj_constants = [i['cost_function_constant'] for i in loaded_files]
con_qubos = [i['constraint_function_qubo'] for i in loaded_files]
con_constants = [i['constraint_function_constant'] for i in loaded_files]

# Prepare the data

Here we already get the qubos and constraints. So we only need to calculate penalties and get the full QUBOs.

In [3]:
minimisation = True # This is a minimisation problem
QUBOs, penalties = Experiment.data_prep_light(obj_qubos, con_qubos, 'Minimum Lazy', minimisation)
qubo_sizes = [max(qubo, key=tuple)[0] + 1 for qubo in QUBOs]

# Run experiments

In [4]:
# The number of samples we want the sampler to return
repeats = 20

## Greedy

In [5]:
greedy_sampler = greedy.SteepestDescentSampler()
greedy_runs = Experiment.run_sampler(QUBOs, obj_qubos, obj_constants, con_qubos, con_constants, 
                                     greedy_sampler, repeats, num_reads=150)

100.0 %


## Simulated Annealing

In [6]:
sa_sampler = neal.SimulatedAnnealingSampler()
sa_runs = Experiment.run_sampler(QUBOs, obj_qubos, obj_constants, con_qubos, con_constants, 
                                 sa_sampler, repeats, num_reads=30)

100.0 %


## Tabu

In [7]:
tabu_sampler = tabu.TabuSampler()
tabu_runs = Experiment.run_sampler(QUBOs, obj_qubos, obj_constants, con_qubos, con_constants, 
                                   tabu_sampler, repeats, timeout=2700)

100.0 %


# Record the results

In [8]:
greedy_results = Table.record_results(greedy_runs, qubo_sizes, penalties, repeats, minimisation)
sa_results = Table.record_results(sa_runs, qubo_sizes, penalties, repeats, minimisation)
tabu_results = Table.record_results(tabu_runs, qubo_sizes, penalties, repeats, minimisation)

# Display the first repetition table
rep = 0
Table.display_side_by_side(greedy_results[rep], sa_results[rep], tabu_results[rep], titles=['Greedy', 'SA', 'Tabu'])

,Size,Penalty,Objective Function,Broken Constraints,Energy (minimisation)
0,144,5460,1756,0,1756
1,196,8968,2900,0,2900
2,256,12580,3892,0,3892
3,324,16102,5566,0,5566
4,400,20928,7290,0,7290
5,144,874944,256048,0,256048
6,225,1498176,386990,0,386990
7,400,2569174,804246,0,804246
8,1600,10418804,3448552,0,3448552
9,1600,4524144275,756679343,0,756679343


# Explore the results

In [9]:
# Show total energies of all tries in all problems in a single df
energies_greedy = Table.columns_to_table(greedy_results, 'Energy (minimisation)')
energies_sa = Table.columns_to_table(sa_results, 'Energy (minimisation)')
energies_tabu = Table.columns_to_table(tabu_results, 'Energy (minimisation)')

energies_tabu

,Energy (minimisation) 0,Energy (minimisation) 1,Energy (minimisation) 2,Energy (minimisation) 3,Energy (minimisation) 4,Energy (minimisation) 5,Energy (minimisation) 6,Energy (minimisation) 7,Energy (minimisation) 8,Energy (minimisation) 9,Energy (minimisation) 10,Energy (minimisation) 11,Energy (minimisation) 12,Energy (minimisation) 13,Energy (minimisation) 14,Energy (minimisation) 15,Energy (minimisation) 16,Energy (minimisation) 17,Energy (minimisation) 18,Energy (minimisation) 19
0,1676,1692,1692,1672,1676,1676,1690,1680,1682,1692,1688,1676,1680,1686,1682,1690,1690,1690,1686,1684
1,2786,2794,2772,2766,2798,2756,2780,2792,2772,2796,2798,2756,2788,2784,2792,2778,2790,2782,2778,2796
2,3814,3794,3818,3808,3786,3798,3814,3778,3804,3792,3804,3800,3820,3770,3822,3778,3796,3780,3790,3810
3,5490,5458,5486,5442,5488,5490,5474,5456,5494,5472,5482,5490,5496,5500,5456,5488,5486,5512,5500,5474
4,7118,7060,7112,7072,7106,7094,7088,7146,7050,7098,7056,7110,7116,7080,7026,7066,7122,7072,7122,7056
5,245564,240522,240630,245104,249510,244668,245718,241844,235654,243568,246102,244984,246720,243392,244674,242410,245902,242820,246850,244684
6,373188,378934,382020,379118,379636,374180,376494,379170,380922,372452,377580,381090,372592,378654,378032,378870,377806,378702,374708,375172
7,789500,786288,778902,789066,785726,763552,781420,784676,782928,776890,783052,775318,777900,782422,782384,766226,762576,766652,780298,774028
8,3365752,3359400,3347932,3361268,3362472,3374456,3368072,3354340,3346488,3326188,3348332,3361468,3364392,3350224,3358586,3375548,3355758,3367862,3354750,3346954
9,700262176,697877189,713953950,690105981,723333648,725170911,700709678,721915411,711566727,716384654,689257768,684756661,707136934,686907664,727268901,704742589,704742589,726902721,695913230,694947539


In [10]:
# Show number of broken constraints of all tries in all problems in a single df
broken_constraints_greedy = Table.columns_to_table(greedy_results, 'Broken Constraints')
broken_constraints_sa = Table.columns_to_table(sa_results, 'Broken Constraints')
broken_constraints_tabu = Table.columns_to_table(tabu_results, 'Broken Constraints')

broken_constraints_greedy

,Broken Constraints 0,Broken Constraints 1,Broken Constraints 2,Broken Constraints 3,Broken Constraints 4,Broken Constraints 5,Broken Constraints 6,Broken Constraints 7,Broken Constraints 8,Broken Constraints 9,Broken Constraints 10,Broken Constraints 11,Broken Constraints 12,Broken Constraints 13,Broken Constraints 14,Broken Constraints 15,Broken Constraints 16,Broken Constraints 17,Broken Constraints 18,Broken Constraints 19
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
# Show feasible solutions
feasible_full_greedy = Table.feasibility_table(greedy_results)
feasible_full_sa = Table.feasibility_table(sa_results)
feasible_full_tabu = Table.feasibility_table(tabu_results)

feasible_full_tabu

,Feasible 0,Feasible 1,Feasible 2,Feasible 3,Feasible 4,Feasible 5,Feasible 6,Feasible 7,Feasible 8,Feasible 9,Feasible 10,Feasible 11,Feasible 12,Feasible 13,Feasible 14,Feasible 15,Feasible 16,Feasible 17,Feasible 18,Feasible 19
0,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
5,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
6,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
7,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
8,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
9,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True


This is the final and the most important table. It displays the number of feasible solutions achieved for every problem, the feasibility rate (feasible_soultions/total_solutions), the mean energy of the solutions and the standard deviation.

In [12]:
# Calculate number of feasible solutions with mean and SD (in all runs)
feasible_greedy = Table.feasibility_statistic(greedy_results)
feasible_sa = Table.feasibility_statistic(sa_results)
feasible_tabu = Table.feasibility_statistic(tabu_results)

# Display the table
Table.display_side_by_side(feasible_greedy, feasible_sa, feasible_tabu, titles=['Greedy', 'SA', 'Tabu'])

,Feasible,Feasibility rate,Energy mean,Energy SD
0,20.0,1.0,1.740900e+03,1.657487e+01
1,20.0,1.0,2.867800e+03,3.833866e+01
2,20.0,1.0,3.897400e+03,2.212976e+01
3,20.0,1.0,5.595800e+03,4.071932e+01
4,20.0,1.0,7.274300e+03,3.265304e+01
5,20.0,1.0,2.571473e+05,3.773328e+03
6,20.0,1.0,3.939724e+05,6.057838e+03
7,20.0,1.0,8.018588e+05,7.603917e+03
8,20.0,1.0,3.455346e+06,1.610131e+04
9,20.0,1.0,7.576038e+08,1.385878e+07


# Save results

In [13]:
data_folder = '../Data/Produced/Quadratic Assignment Problem/'
broken_constraints_greedy.to_pickle(data_folder + 'lazy_qap_greedy_broken_constraints.pkl')
broken_constraints_sa.to_pickle(data_folder + 'lazy_qap_sa_broken_constraints.pkl')
broken_constraints_tabu.to_pickle(data_folder + 'lazy_qap_tabu_broken_constraints.pkl')

feasible_greedy.to_pickle(data_folder + 'lazy_qap_greedy_feasible.pkl')
feasible_sa.to_pickle(data_folder + 'lazy_qap_sa_feasible.pkl')
feasible_tabu.to_pickle(data_folder + 'lazy_qap_tabu_feasible.pkl')